
### Done
* Forward pass
* Backwards pass
* Summaries
* Replay buffer
* target network
* Save model
* Breakout 
* preprocessing


### Missing
* Action repeats
* TensorBoard Summaries
    * Track episode lengths!
* Set max episode length! (double of the mean?)
* Let it train
* ? use int8 to store the experiences? ~ x4 memory saver

### Notes
Inspired by: https://medium.com/@awjuliani/simple-reinforcement-learning-with-tensorflow-part-4-deep-q-networks-and-beyond-8438a3e2b8df

# DQN on Breakout



In [ ]:
%%javascript

Jupyter.keyboard_manager.command_shortcuts.add_shortcut('r', {
    help : 'run all cells',
    help_index : 'zz',
    handler : function (event) {
        IPython.notebook.execute_all_cells();
        return false;
    }
});

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

%matplotlib inline
import os
import sys
import shutil
import numpy as np
import time
import matplotlib.pyplot as plt

import tensorflow as tf
import gym

from tensorflow.contrib.keras.api.keras.layers import Dense, Input, Conv2D, Flatten
from tensorflow.contrib.keras.api.keras.models import Model

import utils

In [ ]:
## Hyperparameters
update_frequency = 4
batch_size = 16
training_time = int(50e6) # frames
target_size = [84,84] #List of integer
num_frames = 4 # Integer
gray = True # Make obs grayscale 

replay_buffer_size = int(1e6) # DOESN'T FIT in memory
replay_buffer_size = int(2e5)
minimum_experience = replay_buffer_size

learning_rate = 0.0001
gamma = 0.99
epsilon_init = 1.0
epsilon_end = 0.1

annealing_period = 1e6 # frames
# annealing_period = 5e5 # frames

In [ ]:
## Parameters
if 1:
    env_name = 'Breakout-v0'
    model_type = 'DQN'
    path = "./dqn" #The path to save our model to.
    
else:
    env_name = 'CartPole-v1'
    model_type = 'dense'
    path = './dense'

In [ ]:
## Derived settings
tf.reset_default_graph()
annealer = utils.Annealer(epsilon_init, epsilon_end, annealing_period)

env = gym.make(env_name)
obs = env.reset()
s_size = list(obs.shape)
a_size = env.action_space.n
n_channels = 1 if gray else 4
target_dim = target_size + [num_frames*n_channels]
print('-- Environmental variables --')
print('env_name  ', env_name)
print('model_type', model_type)
print('s_size    ', s_size)
print('a_size    ', a_size)
print('target_dim', target_dim)

obsPlaceholder = tf.placeholder(tf.float32, shape=[None]+target_dim, name='obsPlaceholder')
obsPlaceholder = tf.placeholder(tf.float32, shape=[None]+target_dim, name='obsPlaceholder')
# Assume action is encoded as ONE number
actionPlaceholder = tf.placeholder(tf.int32, shape=[None], name='actionPlaceholder')
targetQPlaceholder = tf.placeholder(tf.float32, shape=[None], name='targetQPlaceholder')

print('\n-- Placeholders --')
print('obsPlaceholder    ', obsPlaceholder.get_shape())
print('actionPlaceholder ', actionPlaceholder.get_shape())
print('targetQPlaceholder', targetQPlaceholder.get_shape())
print()

### Create model
from models import DQN
with tf.name_scope('DQN1'):
    DQN1 = DQN(model_type, obsPlaceholder, actionPlaceholder, a_size)
    DQN1.create_MSE_train_op(targetQPlaceholder, learning_rate=learning_rate)

with tf.name_scope('DQN2'):
    DQN2 = DQN(model_type, obsPlaceholder, actionPlaceholder, a_size)
    DQN2.create_MSE_train_op(targetQPlaceholder, learning_rate=learning_rate)

print('Model summary')
DQN1.model.summary()

In [ ]:
## Preprocessor
preprocessor = utils.DataHandler()
preprocessor.define_preprocess_2d(target_size=target_size, num_frames=num_frames, gray=gray)


In [ ]:
## Stuff for saving the model
saver = tf.train.Saver()

In [ ]:
## Script specific helper functions

## Stuff for updating the target graph
def updateTargetGraph(tfVars,tau):
    # TODO: FIX THIS
    # This is a terrible way of doing it, it relies on the order in which the networks were created
    total_vars = len(tfVars)
    op_holder = []
    for idx,var in enumerate(tfVars[0:total_vars//2]):
        op_holder.append(tfVars[idx+total_vars//2].assign((var.value()*tau) + ((1-tau)*tfVars[idx+total_vars//2].value())))
    return op_holder


def updateTarget(op_holder,sess):
    for op in op_holder:
        sess.run(op)

tau = 0.001 #Rate to update target network toward primary network
trainables = tf.trainable_variables()
targetOps = updateTargetGraph(trainables,tau)

In [ ]:
buffer = utils.Experience_buffer(replay_buffer_size)
f = 0
training_summaries = {}
training_summaries['num_ep'] = 0
training_summaries['ep_rewards'] = []
training_summaries['epsilon'] = []

load_model = False

In [ ]:
load_model = True

In [ ]:
from datetime import datetime
def current_time():
    return '{:2}:{:<2}'.format(datetime.now().hour, datetime.now().minute)

In [ ]:
assert update_frequency > 1, "The dimensions doen't allign if update_frequency" \
    + 'is less than 1: ' + str(update_frequency)


try:
    print('Session restarted')
    sess.close()
except:
    pass
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# TODO: Use TF summaries, this is shit

if load_model == True:
    print('Loading Model...')
    ckpt = tf.train.get_checkpoint_state(path)
    try:
        saver.restore(sess, ckpt.model_checkpoint_path)
    except AttributeError:
        if not os.path.exists(path):
            os.mkdir(path)
        print('WARNING: Could not load previous model')

else:
    if os.path.exists(path):
        print('Deleting old model')
        shutil.rmtree(path)
    os.mkdir(path)

preprocessor.reset_buffer_2d()
obs, ep_t, ep_r = new_episode(env)
preprocessor.add_2d(obs)
obs_input = preprocessor.get_buffer_2d()

while True:
    try:
        if f == minimum_experience:
            print('___/// BEGIN TRAINING \\\___')
        epsilon = annealer.linear(f-minimum_experience)
        action = sess.run(DQN1.action, feed_dict={obsPlaceholder : [obs_input]}) \
            if np.random.rand(1) > epsilon else np.random.randint(a_size)
        action = int(action)
#         print('action', type(action), action)
        assert 'int' in str(type(action)), 'action must be an int, not a ' + str(type(action))
        reward = 0
        for i in range(4):
            next_obs, r, done, _ = env.step(action)
            r = np.clip(r, -1, 1)
            reward += r
            if done:
                break
        preprocessor.add_2d(next_obs)
        next_obs_input = preprocessor.get_buffer_2d()

        # TODO: wrap in action repeater
        ep_t += 1
        ep_r += reward
        if 'CartPole' in env_name and done and ep_t < 400 :
            # Necessary help for he Q-network to avoid death
            reward = -1

        experience = {'obs':[obs_input], 'action':action, 'reward':reward,
                      'next_obs':[next_obs_input], 'done':done}
        buffer.add(experience)
        obs = next_obs
        
        ## Update Weights
        if f % update_frequency == 0 \
                and f > minimum_experience:
            train_batch = buffer.sample(batch_size)

            # Compute ... TODO: add description
            [action] = sess.run([DQN1.action],
                feed_dict={obsPlaceholder : train_batch['next_obs']})
            [actualQs] = sess.run([DQN2.Qout],
                feed_dict={obsPlaceholder : train_batch['next_obs']})

            actualQ = actualQs[range(batch_size), action] # The Q value of the chosen action
            zero_if_done = train_batch['done']*(-1) + 1 # Used to remove the actualQ
                                                        # When at terminal state
            target = train_batch['reward'] + gamma*actualQ*zero_if_done

            ## Update DQN1
            DQN1_train_dict = {
                        obsPlaceholder : train_batch['obs'],
                        actionPlaceholder : train_batch['action'],
                        targetQPlaceholder : target # TODO: Should be target network!
                }
            _ = sess.run(DQN1.train_op, feed_dict=DQN1_train_dict)
            
            ## Update DQN2
            updateTarget(targetOps,sess)
        
        ## Track training
        track_interval = 5000
        if f % track_interval == 0:
            if f % (track_interval*10) == 0:
                print('\n{:5} {:>9}  {:>7}  {:>7}  {:>7}'.format(
                    'time', 'frames', 'epis', 'reward', 'epsilon'))
            else:
                print('{:5} {:9}, {:7}, {:7.1f}, {:7.3f}'.format(
                      current_time(),
                      f, 
                      training_summaries['num_ep'],
                      np.mean(training_summaries['ep_rewards'][-100:]),
                      training_summaries['epsilon'][-1],
                     ), end='')
            if f % (track_interval*10) == 0 and f>0:
                i = training_summaries['num_ep']
                model_save_str = path+'/model-'+str(i)+'.cptk'
                saver.save(sess, model_save_str)
                print("Saved Model: " + model_save_str)
            else:
                print()
        
        if training_summaries['num_ep'] % 100 == 0:
            env.render()

        if done:
            training_summaries['num_ep'] += 1
            training_summaries['ep_rewards'].append(ep_r)
            training_summaries['epsilon'].append(epsilon)
            preprocessor.reset_buffer_2d()
            obs, ep_t, ep_r = new_episode(env)
            preprocessor.add_2d(obs)
            obs_input = preprocessor.get_buffer_2d()
        f += 1
        
            
    except KeyboardInterrupt:
        break
        

env.render(close=True)
print('\nTerminated')

In [ ]:
## Some memory evaluation, that doesn't seem to work...
def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

experience_size = 0
experience_size += sys.getsizeof(buffer.buffer[0]['action'])
experience_size += sys.getsizeof(buffer.buffer[0]['done'])
experience_size += sys.getsizeof(buffer.buffer[0]['obs'][0])
experience_size += sys.getsizeof(buffer.buffer[0]['next_obs'][0])

total_ram = 30 * 1024 * 1024 * 1024
buffer_len = buffer.buffer_size()

print('buffer len', buffer_len)
print('experience_size', sizeof_fmt(experience_size))
print('buffer memory', sizeof_fmt(experience_size*buffer_len))
print('total ram', sizeof_fmt(total_ram))
print('buffer max', sizeof_fmt(experience_size * replay_buffer_size))
print('Deepmind memory', sizeof_fmt(experience_size * 1e6))

print(total_ram)

In [ ]:
buffer.buffer[0]['next_obs'][0].shape

In [ ]:
class EnvironmentInterface():
    def __init__(self, action_repeats=1, merge_frames=False):
        self.action_repeats = action_repeats
        self.merge_frames = merge_frames
    
    def take_action(self, action, preprocessor=None, epsilon=None):
    
        if self.merge_frames:
            assert self.action_repeats > 1, 'ERROR: EnvironmentInterface: '\
                + 'Cannot merge frames with action_repeats !> 1.'\
                + 'action_repeats = ' + str(self.action_repeats)
                
        if self.epsilon is not None:
            pass
    
    def reset_env(self, env, preprocessor=None):
        """Simple wrapper that restarts the environment"""
        obs = env.reset()
        episode_time_step = 0
        episode_reward = 0
        return obs, episode_time_step, episode_reward

envInter = EnvironmentInterface(env, action_repeats=4, merge_frames=True)
env.take_action(1, preprocessor)

###
env = gym.make(env_name)
obs = env.reset()
preprocessor.reset_buffer_2d()
preprocessor.add_2d(obs)
obs_input = preprocessor.get_buffer_2d()

###


In [ ]:
## Run and render a forward pass
import time

env = gym.make(env_name)
obs = env.reset()
preprocessor.reset_buffer_2d()
preprocessor.add_2d(obs)
obs_input = preprocessor.get_buffer_2d()

env.render()
reward_sum = 0



while True:
    try:
        action = sess.run(DQN1.action, feed_dict={obsPlaceholder : [obs_input]}) \
            if np.random.rand(1) > 0.05 else np.random.randint(a_size)
        action = int(action)
        assert 'int' in str(type(action))
        time.sleep(0.025)
        reward = 0
        for i in range(4):
            obs, r, done, _ = env.step(action)
            r = np.clip(r, -1, 1)
            reward += r
            if done:
                break
        preprocessor.add_2d(obs)
        obs_input = preprocessor.get_buffer_2d()

        env.render()
        reward_sum += reward
        if done:
            print(reward_sum)
            reward_sum = 0
            obs = env.reset()
            preprocessor.reset_buffer_2d()
            preprocessor.add_2d(obs)
            obs_input = preprocessor.get_buffer_2d()

    except KeyboardInterrupt:
        break

# env.render(close=True)
print('Terminated')

In [ ]:
env.render(close=True)


<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>